# Ejercicio 1: Lanzamiento trucado


Se lanza una moneda n = 10 veces y se encuentra que r = 7 veces cae cara. Usando el algoritmo de Metrópolis:

   a) Encuentre el parámetro asociado a la probabilidad de éxito ˆp de la distribución binomial, es decir, encuentre el máximo de la distribución posterior.

   b) Usando la varianza binomial y los cuantiles de la distribución posterior, encuentre los errores asociados al parámetro $\hat{p}$ a un nivel de confianza del 68% (i.e., $σ^+ - σ^-$).
   
   c) ¿Podemos decir que la moneda está truncada?

Utilice la siguiente distribución a priori para el espacio de parámetros:

$$
\pi(p) = \begin{cases} 
            1 & \text{si } 0 < p < 1 \\
            0 & \text{si otro caso}
         \end{cases}
         \quad (6.90)
$$


# Solucion

In [1]:
import numpy as np
import scipy.stats as stats

In [2]:
# Observed data
n = 10  # Number of coin tosses
r = 7  # Number of times it lands heads

# Prior distribution
prior = stats.uniform(loc=0, scale=1)


In [3]:
# Likelihood function
def likelihood(p):
    return stats.binom.pmf(r, n, p)


# Posterior distribution
def posterior(p):
    return likelihood(p) * prior.pdf(p)


# Metropolis algorithm
def metropolis(n_iterations, initial_value):
    samples = [initial_value]
    current_value = initial_value

    for _ in range(n_iterations):
        # Propose a new value based on a normal distribution
        proposed_value = np.random.normal(current_value, 0.1)

        # Accept or reject the proposed value
        acceptance_ratio = posterior(proposed_value) / posterior(current_value)
        acceptance_prob = min(1, acceptance_ratio)
        if np.random.uniform() < acceptance_prob:
            current_value = proposed_value

        samples.append(current_value)

    return samples



In [4]:
n_iterations = 10000
initial_value = 0.5
samples = metropolis(n_iterations, initial_value)

# Extract posterior distribution statistics
maximum_a_posteriori = np.argmax(samples)


# Print the results
print("Posterior max:", round(maximum_a_posteriori, 5))


Posterior max: 3112


In [5]:
posterior_std = np.std(samples)
quantiles = np.percentile(samples, [16, 84])  # 68% confidence interval

print("Posterior standard deviation:", round(posterior_std,5))
print("68% Confidence interval:", quantiles)


Posterior standard deviation: 1.35144
68% Confidence interval: [0.65569138 2.94670171]


In [6]:
# Check if the posterior mean is significantly different from 0.5
posterior_mean = np.mean(samples)
is_biased = np.abs(posterior_mean - 0.5) > 0.1

if is_biased:
    print("The coin is biased.")
else:
    print("The coin is not biased.")

The coin is biased.
